In [24]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# Cargar el dataset
crops = pd.read_csv("soil_measures.csv")

# Mostrar las primeras filas del DataFrame
print("\n", crops.head(15))
print("\n", crops.describe())

# Comprobar valores faltantes
print("\n", crops.isna().sum())

# Compruebe cuántos cultivos tenemos, es decir, objetivo de clases múltiples
print("\n", crops.crop.unique())
print("\n")

# Dividir en conjuntos de características y objetivos
X = crops.drop(columns="crop")
y = crops["crop"]

# Divida los datos en conjuntos de entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Cree un diccionario para almacenar el rendimiento del modelo para cada característica.
feature_performance = {}

# Entrene un modelo de regresión logística para cada característica
for feature in ["N", "P", "K", "ph"]:
    log_reg = LogisticRegression(multi_class="multinomial", max_iter=200, solver='lbfgs')
    log_reg.fit(X_train_scaled[:, [X.columns.get_loc(feature)]], y_train)
    y_pred = log_reg.predict(X_test_scaled[:, [X.columns.get_loc(feature)]])
    
    # Calcule la puntuación F1, la media armónica de precisión y recuperación
    f1 = metrics.f1_score(y_test, y_pred, average="weighted")
    
    # Agregue pares de puntuación característica-f1 al diccionario
    feature_performance[feature] = f1
    print(f"F1-score for {feature}: {f1}")

# K obtuvo la mejor puntuación en F1
# Almacenar en el diccionario best_predictive_feature
best_predictive_feature = {"K": feature_performance["K"]}
print("\n Mejor característica predictiva:", best_predictive_feature)



      N   P   K        ph  crop
0   90  42  43  6.502985  rice
1   85  58  41  7.038096  rice
2   60  55  44  7.840207  rice
3   74  35  40  6.980401  rice
4   78  42  42  7.628473  rice
5   69  37  42  7.073454  rice
6   69  55  38  5.700806  rice
7   94  53  40  5.718627  rice
8   89  54  38  6.685346  rice
9   68  58  38  6.336254  rice
10  91  53  40  5.386168  rice
11  90  46  42  7.502834  rice
12  78  58  44  5.108682  rice
13  93  56  36  6.984354  rice
14  94  50  37  6.948020  rice

                  N            P            K           ph
count  2200.000000  2200.000000  2200.000000  2200.000000
mean     50.551818    53.362727    48.149091     6.469480
std      36.917334    32.985883    50.647931     0.773938
min       0.000000     5.000000     5.000000     3.504752
25%      21.000000    28.000000    20.000000     5.971693
50%      37.000000    51.000000    32.000000     6.425045
75%      84.250000    68.000000    49.000000     6.923643
max     140.000000   145.000000   20

## Conclusiones

La característica N (nitrógeno) tiene una puntuación F1 de aproximadamente 0.107. Esto indica que, cuando se usa solo N para predecir el tipo de cultivo, el modelo tiene un rendimiento bajo.

La característica P (fósforo) tiene una puntuación F1 de aproximadamente 0.084, lo que también indica un rendimiento bajo.

La característica K (potasio) tiene la puntuación F1 más alta entre las evaluadas, con aproximadamente 0.138. Aunque es la mejor característica predictiva entre las cuatro, sigue siendo relativamente baja en términos absolutos.

La característica ph tiene la puntuación F1 más baja, con aproximadamente 0.045. Esto indica que es la menos efectiva de las cuatro características para predecir el tipo de cultivo.

Entre las características evaluadas, K (potasio) es la mejor en términos de su capacidad predictiva, pero su puntuación F1 sigue siendo baja.

### Modelo combinando caracteristicas 


In [25]:
log_reg = LogisticRegression(multi_class="multinomial", max_iter=200, solver='lbfgs')
log_reg.fit(X_train_scaled, y_train)
y_pred = log_reg.predict(X_test_scaled)
f1 = metrics.f1_score(y_test, y_pred, average="weighted")
print(f"F1-score with all features: {f1}")


F1-score with all features: 0.6475209891596989


Obtener una puntuación F1 de aproximadamente 0.648 utilizando todas las características combinadas es un resultado significativamente mejor comparado con las puntuaciones individuales de las características. Esto indica que el modelo de regresión logística puede capturar mejor la relación entre las características y la variable objetivo cuando tiene acceso a todas las características en lugar de solo una.

La puntuación F1 de 0.648 sugiere que el modelo está haciendo un trabajo razonable en la clasificación de los tipos de cultivos. No es perfecta, pero es una mejora considerable respecto a usar características individuales.